In [51]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering
using BenchmarkTools, Compat
using DelimitedFiles
using SparseArrays
using LinearAlgebra
using SphericalGraphEmbedding
using Random

In [52]:
dataset = "PolBlogs"
dataset = "PowerEU"
dataset = "Facebook"
dataset = "PowerUS"
dataset = "HEP"
# dataset = "Internet"
# dataset = "CondMat2003"
#dataset = "Pokec"

# Initialization
A = 0
r = 0
dim0 = 0
community = zeros(Int64,N,1);

# nb iterations projected power method
n_it_PPM = 30000

# relative objective variation
t = 1e-8

# nb of times vector partition is repeated
n_it_vec_part = 10 

# nb updates vec part
n_updates = 50 

# shape of embedding
shape = "Ellipsoidal"

# Loading data

if dataset == "PolBlogs"

    A0 = readdlm("data/PolBlogs/PolBlogsAdj.txt")
    lab = readdlm("data/PolBlogs/LabelsPolBlogs.txt")

    dim0 = size(A0)
    A  = zeros(Int64 ,dim0[1],dim0[1])

    for i = 1:dim0[1]
        for j = 1:dim0[1]
            A[i,j] = Int64(A0[i,j])
        end
    end
    A = A+A'
    A = sparse(A)

    A,_ = largest_component(A)

    A0 = 0
    N = size(A,2)

    # number of columns of initial guess
    d0 = 20
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerEU"

    list = readdlm("data/PowerEU/ElistPowerEU.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Facebook"

    list = readdlm("data/Facebook/facebook-gender_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # here start at 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) # here start at 0

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerUS"

    list = readdlm("data/PowerUS/ElistPowerUS.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "HEP"

    list = readdlm("data/HEP/hep-year_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Wikivote"

    list = readdlm("data/Wikivote/wiki-Vote.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Internet"

    list = readdlm("data/Internet/InternetEList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "CondMat2003"

    list = readdlm("data/CondMat2003/CondMat2003EList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    p = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Pokec"

    list = readdlm("data/Pokec/soc-pokec-relationships.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,_ = largest_component(A0)

    A0 = 0
    M_int = 0

    # number of columns of initial guess
    d0 = 50
    # number of centroids thrown in embedding
    n_clusters = 100

end

100

In [53]:
print(dataset)
print("\n")
x_embed,community,singular =  @time sphere_embed_cluster(A, n_it_PPM, t, n_clusters, n_it_vec_part, n_updates, shape, d0)
# run twice to get timings without compilation time

HEP
 ------- Acc Projected Power Iteration -------
The iteration has become stationary after 249 iterations
 53.210793 seconds (7.39 k allocations: 22.911 GiB, 8.79% gc time)
 -------------- Clustering ------- 
dimension of embedding used for clustering: 50
Number of updates: 31
Number of communities: 11
Modularity: 0.6133639058902282
 -------------------------------------------- 
The first 5 squared singular values divided by N : 
[0.5271776686933346, 0.24814214611090998, 0.16065312735143128, 0.042906295122825656, 0.021114413467887044]
 -------------------------------------------- 
166.487245 seconds (23.11 M allocations: 40.280 GiB, 5.70% gc time)


([-0.004880901357709299 -0.0023617526361155057 … -0.006288391837689974 0.002015788584667445; -0.008363477093580873 0.009701060253581221 … -0.0040273875070100775 -0.005129423613485266; … ; -0.0002745801859200701 0.0008187338510736741 … -0.0003784434459069713 0.0009359916802831991; 0.0013885365364893017 -0.00016294393416883346 … 0.00011992063986503726 -0.0004341538541014601], [1, 2, 2, 3, 4, 3, 3, 3, 3, 5  …  5, 3, 3, 2, 6, 3, 3, 3, 6, 6], [120.18597306756462, 82.45662376934271, 66.34678356506227, 34.28749752264552, 24.052753044508336, 0.38514994410141057, 0.12739564397158687, 0.06545081729719546, 0.05035843715719228, 0.030325260262961944  …  0.0003840811753376466, 0.000354191516882918, 0.00030895394312118845, 0.000200895544448172, 0.00015782120813048097, 0.0001199197963524278, 0.0001148995708830136, 9.119101720972526e-5, 5.376595029225893e-5, 4.9663774777911216e-5])